'telecom.csv'

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
df=pd.read_csv('telecom.csv')

data_x=df[df.columns.difference(['Churn','customerID','TotalCharges'])]

y=df.Churn

In [14]:
y.isin(['Yes']).sum(axis=0), y.isin(['No']).sum(axis=0)

(1583, 4417)

In [15]:
#change string columns
data_x=pd.get_dummies(data_x, columns=['DeviceProtection','InternetService','OnlineSecurity','OnlineBackup','StreamingTV','StreamingMovies','TechSupport'],drop_first=True)
# data_x.drop(columns=['DeviceProtection_No internet service','OnlineSecurity_No internet service','OnlineBackup_No internet service','StreamingTV_No internet service','StreamingMovies_No internet service','TechSupport_No internet service'], inplace=True)
data_x=pd.get_dummies(data_x, columns=['gender','Dependents','PaperlessBilling','Partner','PhoneService','MultipleLines'],drop_first=True)
# data_x.drop(columns=['MultipleLines_No phone service'],inplace=True)
data_x=pd.get_dummies(data_x, columns=['Contract','PaymentMethod'])
y=pd.get_dummies(y, drop_first=True)

Split data into 2 parts, take the small one as the final test. You will be using for each model's final evaluation

In [16]:
x_cur, x_sep, y_cur, y_sep=train_test_split(data_x, y, test_size=0.1, random_state=5)


In [17]:
len(x_cur), len(x_sep), len(y_cur), len(y_sep)

(5400, 600, 5400, 600)

# Build 5 classification models

logistic regression, decision tree, random forest, knn, svm

#### tips
<br>
1. use cross validation
<br>
2. use same splits for all classifiers (to have correct comparison)
<br>
3. build a table to show statistics for all classifiers (at least - accuracy, precision and recall. Any more component would be a +. For train - at least accuracy)

In [18]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import statistics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from imblearn.over_sampling import SMOTE


In [19]:
n='0'

metrics_list=[]

metrics_for=['logreg','dectree','randforest','knn','svm']

types_for=['train','test', 'sep_train','sep_test' ]

logreg=LogisticRegression()
clf = DecisionTreeClassifier(max_depth=5)
rclf = RandomForestClassifier(n_estimators=300,max_depth=4)
neigh = KNeighborsClassifier(n_neighbors=3)
svmclf = svm.SVC(kernel='linear')

sm = SMOTE() #make yes rows as many as no rows (we have imbalanced data)

kf = KFold(n_splits=10)
#do cross validation
for train_index, test_index in kf.split(x_cur):
    x_train, x_test = x_cur.iloc[train_index], x_cur.iloc[test_index]
    y_train, y_test = y_cur.iloc[train_index], y_cur.iloc[test_index]
    
    for tp in metrics_for:
        
        pred_train=0
        pred_test=0

        if (tp=='logreg'):
            
            
            x_train_os, y_train_os = sm.fit_sample(x_train, y_train)
        
            logreg.fit(x_train_os,y_train_os)
    
            pred_train=logreg.predict(x_train_os)
            pred_test=logreg.predict(x_test)
            
            n='1'
            
        elif (tp=='dectree'):
   
            x_train_os, y_train_os = sm.fit_sample(x_train, y_train)
                                                   
            clf = clf.fit(x_train_os,y_train_os)

            pred_train = clf.predict(x_train_os)
            pred_test = clf.predict(x_test)
            
            n='2'  
        elif(tp=='randforest'):
                                                   
            x_train_os, y_train_os = sm.fit_sample(x_train, y_train)
            rclf.fit(x_train_os,y_train_os)
            pred_train = rclf.predict(x_train_os)
            pred_test = rclf.predict(x_test)
            
            n='3' 
        elif(tp=='knn'):
            x_train_os, y_train_os = sm.fit_sample(x_train, y_train)
            neigh.fit(x_train_os,y_train_os)
            pred_train = neigh.predict(x_train_os)
            pred_test = neigh.predict(x_test)
            
            n='4'
        elif(tp=='svm'):
            
            x_train_os, y_train_os = sm.fit_sample(x_train, y_train)
            svmclf.fit(x_train_os,y_train_os)
            
            pred_train = svmclf.predict(x_train_os)
            pred_test = svmclf.predict(x_test)
            
            n='5'
            
        metrics_list.append([n+'11',tp+'_train_acc',metrics.accuracy_score(y_train_os, pred_train)])
        metrics_list.append ([n+'21',tp+'_test_acc',metrics.accuracy_score(y_test, pred_test)])

        metrics_list.append([n+'12',tp+'_train_perc',metrics.precision_score(y_train_os, pred_train, average=None)[0]])
        metrics_list.append([n+'22',tp+'_test_perc',metrics.precision_score(y_test, pred_test, average=None)[0]])
        
        metrics_list.append([n+'13',tp+'_train_recall',metrics.recall_score(y_train_os, pred_train)])
        metrics_list.append([n+'23',tp+'_test_recall',metrics.recall_score(y_test, pred_test)])
  


#create final model and test on sep datas

# x_train=x_cur
# y_train=y_cur
x_test=x_sep
y_test=y_sep

x_train, y_train = sm.fit_sample(x_cur, y_cur)

for tp in metrics_for:

    pred_train=0
    pred_test=0

    if (tp=='logreg'):

        logreg.fit(x_train,y_train)

        pred_train=logreg.predict(x_train)
        pred_test=logreg.predict(x_test)

        n='1'

    elif (tp=='dectree'):

        clf = clf.fit(x_train,y_train)

        pred_train = clf.predict(x_train)
        pred_test = clf.predict(x_test)

        n='2'  
    elif(tp=='randforest'):
        rclf.fit(x_train,y_train)
        pred_train = rclf.predict(x_train)
        pred_test = rclf.predict(x_test)
        
        n='3' 
    elif(tp=='knn'):
        neigh.fit(x_train,y_train)
        pred_train = neigh.predict(x_train)
        pred_test = neigh.predict(x_test)

        n='4'
    elif(tp=='svm'):

        svmclf.fit(x_train,y_train)
            
        pred_train = svmclf.predict(x_train)
        pred_test = svmclf.predict(x_test)

        n='5'

    metrics_list.append([n+'31',tp+'_sep_train_acc',metrics.accuracy_score(y_train, pred_train)])
    metrics_list.append ([n+'41',tp+'_sep_test_acc',metrics.accuracy_score(y_test, pred_test)])

    metrics_list.append([n+'32',tp+'_sep_train_perc',metrics.precision_score(y_train, pred_train, average=None)[0]])
    metrics_list.append([n+'42',tp+'_sep_test_perc',metrics.precision_score(y_test, pred_test, average=None)[0]])

    metrics_list.append([n+'33',tp+'_sep_train_recall',metrics.recall_score(y_train, pred_train)])
    metrics_list.append([n+'43',tp+'_sep_test_recall',metrics.recall_score(y_test, pred_test)])
            

metrics_df= pd.DataFrame(metrics_list)
metrics_dfg=metrics_df.groupby([0,1]).mean()


    
statistica=pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall'])

for row in metrics_dfg.iterrows():
    metr=int(row[0][0][0])-1
    tp=int(row[0][0][1])-1
    col_index=int(row[0][0][2])-1
    
    metr_v=metrics_for[metr]
    tp_v=types_for[tp]
   
    col=statistica.columns[col_index]
    
    
  
    value=row[1].values

    
    row=metr_v+'_'+tp_v
  
  
    
    if(row not in statistica.index):    
        statistica.loc[row]=[0,0,0]
  

    statistica[col][row]=value[0]
   
       

In [20]:
statistica

,Accuracy,Precision,Recall
logreg_train,0.839906,0.840776,0.841127
logreg_test,0.787963,0.86944,0.649185
logreg_sep_train,0.841052,0.84584,0.847974
logreg_sep_test,0.801667,0.873563,0.647436
dectree_train,0.815532,0.822134,0.824628
dectree_test,0.767593,0.878811,0.695169
dectree_sep_train,0.80959,0.869369,0.890511
dectree_sep_test,0.721667,0.892351,0.75641
randforest_train,0.793665,0.836712,0.857513
randforest_test,0.74537,0.908102,0.795231


#### choose the best model. Either retrain it or copy it in the cell below

In [22]:
# final_model-RandomForestClassifier

x_train, y_train = sm.fit_sample(x_cur, y_cur)
rclf.fit(x_train,y_train)


RandomForestClassifier(max_depth=4, n_estimators=300)